In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from knowme.tools import SiteAnswerTool, CVAnswerTool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType
import os

In [4]:
site_answer_tool = SiteAnswerTool()
cv_answer_tool = CVAnswerTool()

tools = [site_answer_tool, cv_answer_tool]

In [5]:
agent_message = SystemMessage(
    content=""" You will answer questions from the user about a candidate. 
    The answer might be present in their Notion Website or it might be present in their CV 
    The CV is usually in the pdf format. Your job is to chose the appropriate tool to answer
    the question from. If the information is not present in the website, then consider the CV.     
    """
)
agent_kwargs = {"system_message": agent_message}

In [6]:
llm = ChatOpenAI(
    model="gpt-4", temperature=0, verbose=True, api_key=os.environ["OPENAI_API_KEY"]
)

In [7]:
model_with_tools = llm.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([HumanMessage(content="What is up for dinner?")])
print(f"Content string {response.content}")
print(f"Tool calls: {response.tool_calls}")

Content string I'm sorry, as an AI, I don't have the capability to provide information about personal activities such as dinner plans. I'm here to assist with tasks and provide information within my programming.
Tool calls: []


In [18]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [9]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='7182f80c-50de-4146-9e78-cb8c52a9a2d9'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 139, 'total_tokens': 149}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6eb793b4-86cb-48e1-bddb-40fa80b86b03-0', usage_metadata={'input_tokens': 139, 'output_tokens': 10, 'total_tokens': 149})]

In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What is the news from Abhinav in March 2024")]}
)

response["messages"]

[HumanMessage(content='What is the news from Abhinav in March 2024', id='7b3290b8-9d37-45ab-a11a-bb7bcd46a80a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7SZsErLbDLoXL2dr0agysTy6', 'function': {'arguments': '{\n"query": "What is the news from Abhinav in March 2024",\n"session_id": "123456"\n}', 'name': 'site-answer-tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 150, 'total_tokens': 185}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5568cfd0-350e-4719-bd36-d3f8f5794168-0', tool_calls=[{'name': 'site-answer-tool', 'args': {'query': 'What is the news from Abhinav in March 2024', 'session_id': '123456'}, 'id': 'call_7SZsErLbDLoXL2dr0agysTy6'}], usage_metadata={'input_tokens': 150, 'output_tokens': 35, 'total_tokens': 185}),
 ToolMessage(content='In March 2024, Abhinav Ramesh Kashyap presented their survey paper on sentence representations

In [22]:
response = agent_executor.invoke(
    {
        "messages": [
            HumanMessage(
                content="Which students has abhinav offered mentorship to in his CV?"
            )
        ]
    }
)

response["messages"]

[HumanMessage(content='Which students has abhinav offered mentorship to in his CV?', id='8c0b166d-d7c1-40e3-a180-7d6a9e8e3738'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_34C7v5OwFP9URc5AES7O5nNG', 'function': {'arguments': '{\n  "query": "Which students has abhinav offered mentorship to in his CV?",\n  "session_id": "1234"\n}', 'name': 'cv-answer-tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 151, 'total_tokens': 188}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-246efe20-7321-4555-868c-04b638d1f688-0', tool_calls=[{'name': 'cv-answer-tool', 'args': {'query': 'Which students has abhinav offered mentorship to in his CV?', 'session_id': '1234'}, 'id': 'call_34C7v5OwFP9URc5AES7O5nNG'}], usage_metadata={'input_tokens': 151, 'output_tokens': 37, 'total_tokens': 188}),
 ToolMessage(content='Abhinav Ramesh Kashyap has offered mentorship to t

In [24]:
from langchain.agents import initialize_agent

In [25]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

/Users/abhinavkashyap/Library/Caches/pypoetry/virtualenvs/knowme-Vm6gcGsG-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [26]:
response = agent.run(
    "Which students has abhinav offered mentorship to in the past few years"
)

/Users/abhinavkashyap/Library/Caches/pypoetry/virtualenvs/knowme-Vm6gcGsG-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': '123456'}`


The provided context does not give information about any students that Abhinav Ramesh Kashyap has offered mentorship to in the past few years.
Invoking: `cv-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': '123456'}`


Abhinav Ramesh Kashyap has offered mentorship to Anand Subramanian on his Masters Thesis & Internship in 2023-2024, Benedict on his Masters Thesis in 2021-2022, Li Xinze on his Undergrad Final Year Project in 2019-2020, and both Calvin Tantio and Shi Tian on their Undergrad Final Year Projects in 2018-2019.Abhinav Ramesh Kashyap has offered mentorship to the following students:

1. Anand Subramanian on his Masters Thesis & Internship in 2023-2024
2. Benedict on his Masters Thesis in 2021-2022
3. Li Xinze on his Undergrad

In [28]:
# This works much better than the create-react agent
# This just works on prompts and gpt4 has good capabilities to do the function calling

In [9]:
# lets try creating the tool calling agent instead and see how it works
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You will answer questions from the user about a candidate. 
            Your job is to chose the either the site-answer-tool or cv-answer-tool to answer 
            the question from. You may chose either of the tool first. But if the answer is not present in one use the other and try""",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [10]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
ans = agent_executor.invoke(
    {"input": "Which students has abhinav offered mentorship to in the past few years"}
)



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': '123456'}`


<generator object RunnableBindingBase.stream at 0x319c9ccf0>
Invoking: `cv-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': '123456'}`


<generator object RunnableBindingBase.stream at 0x319c9cba0>I'm sorry, but I couldn't find the information about the students Abhinav has mentored in the past few years.

> Finished chain.


In [45]:
ans["output"]

'Abhinav Ramesh Kashyap has offered mentorship to several students in the past few years. These students include:\n\n- Anand Subramanian for his Masters Thesis & Internship in 2023-2024\n- Benedict for his Masters Thesis in 2021-2022\n- Li Xinze for her Undergrad Final Year Project in 2019-2020\n- Calvin Tantio and Shi Tian for their Undergrad Final Year Projects in 2018-2019.'

In [46]:
for chunk in agent_executor.stream({"input": "Which blog posts has Abhianv written?"}):
    print(chunk)
    print("====================")



> Entering new AgentExecutor chain...
{'actions': [ToolAgentAction(tool='site-answer-tool', tool_input={'query': 'Which blog posts has Abhianv written?', 'session_id': '123456'}, log="\nInvoking: `site-answer-tool` with `{'query': 'Which blog posts has Abhianv written?', 'session_id': '123456'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vHG3BaQgQd5FqMhWnBaQCTxE', 'function': {'arguments': '{\n"query": "Which blog posts has Abhianv written?",\n"session_id": "123456"\n}', 'name': 'site-answer-tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-4bc3550b-1d99-4795-a865-f05edb06b26a', tool_calls=[{'name': 'site-answer-tool', 'args': {'query': 'Which blog posts has Abhianv written?', 'session_id': '123456'}, 'id': 'call_vHG3BaQgQd5FqMhWnBaQCTxE'}], tool_call_chunks=[{'name': 'site-answer-tool', 'args': '{\n"query": "Which blog posts has Abhianv written?",\n"session_id": "123456"\n}', 'id'

In [47]:
agent_executor.invoke({"input": "Hey I am Abhinav"})



> Entering new AgentExecutor chain...
Hello Abhinav! How can I assist you today?

> Finished chain.


{'input': 'Hey I am Abhinav',
 'output': 'Hello Abhinav! How can I assist you today?'}

In [48]:
agent_executor.invoke({"input": "What is my name"})



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'What is my name', 'session_id': '12345'}`


Your name is Abhinav Ramesh Kashyap.Your name is Abhinav Ramesh Kashyap.

> Finished chain.


{'input': 'What is my name', 'output': 'Your name is Abhinav Ramesh Kashyap.'}

In [49]:
agent_executor.invoke({"input": "What did Abhinav do in March 2024"})



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'What did Abhinav do in March 2024', 'session_id': '123456'}`


In March 2024, Abhinav Ramesh Kashyap presented a survey paper on sentence representations at EACL (the European Chapter of the Association for Computational Linguistics).In March 2024, Abhinav Ramesh Kashyap presented a survey paper on sentence representations at EACL (the European Chapter of the Association for Computational Linguistics).

> Finished chain.


{'input': 'What did Abhinav do in March 2024',
 'output': 'In March 2024, Abhinav Ramesh Kashyap presented a survey paper on sentence representations at EACL (the European Chapter of the Association for Computational Linguistics).'}

In [50]:
agent_executor.invoke({"input": "What did Abhinav do in May 2024?"})



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'What did Abhinav do in May 2024?', 'session_id': '123456'}`


In May 2024, Abhinav Ramesh Kashyap's paper titled "M-QALM", a benchmark to test the abilities of clinical large language models, was accepted to the ACL’24 Findings.In May 2024, Abhinav Ramesh Kashyap's paper titled "M-QALM", a benchmark to test the abilities of clinical large language models, was accepted to the ACL’24 Findings.

> Finished chain.


{'input': 'What did Abhinav do in May 2024?',
 'output': 'In May 2024, Abhinav Ramesh Kashyap\'s paper titled "M-QALM", a benchmark to test the abilities of clinical large language models, was accepted to the ACL’24 Findings.'}

In [52]:
agent_executor.invoke(
    {
        "input": "Which students has abhinav offered mentorship to in the past few years. Use session_id=abc for agent calls"
    }
)



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': 'abc'}`


The provided context does not give any information about Abhinav Ramesh Kashyap offering mentorship to any students in the past few years.
Invoking: `cv-answer-tool` with `{'query': 'Which students has abhinav offered mentorship to in the past few years', 'session_id': 'abc'}`


Abhinav Ramesh Kashyap has offered mentorship to the following students in the past few years: Anand Subramanian for his Masters Thesis & Internship in 2023-2024, Benedict for his Masters Thesis in 2021-2022, Li Xinze for his Undergrad Final Year Project in 2019-2020, Calvin Tantio for his Undergrad Final Year Project in 2018-2019, and Shi Tian for his Undergrad Final Year Project in 2018-2019.Abhinav Ramesh Kashyap has offered mentorship to the following students in the past few years: 

1. Anand Subramanian for his Masters Thesi

{'input': 'Which students has abhinav offered mentorship to in the past few years. Use session_id=abc for agent calls',
 'output': 'Abhinav Ramesh Kashyap has offered mentorship to the following students in the past few years: \n\n1. Anand Subramanian for his Masters Thesis & Internship in 2023-2024\n2. Benedict for his Masters Thesis in 2021-2022\n3. Li Xinze for his Undergrad Final Year Project in 2019-2020\n4. Calvin Tantio for his Undergrad Final Year Project in 2018-2019\n5. Shi Tian for his Undergrad Final Year Project in 2018-2019.'}

In [20]:
from knowme.agent import KnowMeAgent

agent = KnowMeAgent(verbose=True)
answer = agent.chat(
    "Which students did Abhinav offere mentorship in the past few years?",
    session_id="abc",
)



> Entering new AgentExecutor chain...

Invoking: `site-answer-tool` with `{'query': 'Which students did Abhinav offere mentorship in the past few years?', 'session_id': 'abc'}`


{'input': 'Which students did Abhinav offere mentorship in the past few years?', 'chat_history': [HumanMessage(content='Which students did Abhinav offere mentorship in the past few years?'), AIMessage(content='The provided context does not include information on any students that Abhinav Ramesh Kashyap may have mentored in the past few years.'), HumanMessage(content='Which students did Abhinav offere mentorship in the past few years?'), AIMessage(content='The provided context does not include information on any students that Abhinav Ramesh Kashyap may have mentored in the past few years.'), HumanMessage(content='Which students did Abhinav offere mentorship in the past few years?'), AIMessage(content='The provided context does not include information on any students that Abhinav Ramesh Kashyap may have mentor

In [23]:
print(answer)

answer.split(" ")

Abhinav Ramesh Kashyap offered mentorship to several students in the past few years. These students include:

1. Anand Subramanian for his Masters Thesis & Internship in 2023-2024
2. Benedict for his Masters Thesis in 2021-2022
3. Li Xinze for his Undergrad Final Year Project in 2019-2020
4. Calvin Tantio for his Undergrad Final Year Project in 2018-2019
5. Shi Tian for his Undergrad Final Year Project in 2018-2019.


['Abhinav',
 'Ramesh',
 'Kashyap',
 'offered',
 'mentorship',
 'to',
 'several',
 'students',
 'in',
 'the',
 'past',
 'few',
 'years.',
 'These',
 'students',
 'include:\n\n1.',
 'Anand',
 'Subramanian',
 'for',
 'his',
 'Masters',
 'Thesis',
 '&',
 'Internship',
 'in',
 '2023-2024\n2.',
 'Benedict',
 'for',
 'his',
 'Masters',
 'Thesis',
 'in',
 '2021-2022\n3.',
 'Li',
 'Xinze',
 'for',
 'his',
 'Undergrad',
 'Final',
 'Year',
 'Project',
 'in',
 '2019-2020\n4.',
 'Calvin',
 'Tantio',
 'for',
 'his',
 'Undergrad',
 'Final',
 'Year',
 'Project',
 'in',
 '2018-2019\n5.',
 'Shi',
 'Tian',
 'for',
 'his',
 'Undergrad',
 'Final',
 'Year',
 'Project',
 'in',
 '2018-2019.']

In [17]:
answer = agent.chat_stream(
    "Which students did Abhinav offere mentorship in the past few years?",
    session_id="abc",
)

In [18]:
for chunk in answer:
    print(chunk)
    print("====")



> Entering new AgentExecutor chain...
{'actions': [ToolAgentAction(tool='site-answer-tool', tool_input={'query': 'Which students did Abhinav offere mentorship in the past few years?', 'session_id': 'abc'}, log="\nInvoking: `site-answer-tool` with `{'query': 'Which students did Abhinav offere mentorship in the past few years?', 'session_id': 'abc'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_z5iJw06BaAUzBvkX9rjm45WZ', 'function': {'arguments': '{\n"query": "Which students did Abhinav offere mentorship in the past few years?",\n"session_id": "abc"\n}', 'name': 'site-answer-tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-26ae2946-4b4f-49a5-8b69-734c08305177', tool_calls=[{'name': 'site-answer-tool', 'args': {'query': 'Which students did Abhinav offere mentorship in the past few years?', 'session_id': 'abc'}, 'id': 'call_z5iJw06BaAUzBvkX9rjm45WZ'}], tool_call_chunks=[{'name': 'site-a

In [19]:
answer["output"]

TypeError: 'generator' object is not subscriptable